## Coding Transformer from scratch in Pytorch

In [ ]:
# Importing "lightning" to make it way easier to write our code  and for automatic code optimization and scaling in the cloud!
! pip3 install lightning

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader

import lightning as L

from position_encoding import PositionEncoding
from attention import Attention
from decoder_only_transformer import DecoderOnlyTransformer

In [3]:
# Prompts: 1. What is Statquest? Awesome 2. Statquest is what? Awesome

token_to_id = {
    "what": 0,
    "is": 1,
    "statquest": 2,
    "awesome": 3,
    "<EOS>": 4
}

In [4]:
id_to_token = {id:token for token, id in token_to_id.items()}

In [5]:
inputs = torch.tensor([[token_to_id["what"],
                        token_to_id["is"],
                        token_to_id["statquest"],
                        token_to_id["<EOS>"],
                        token_to_id["awesome"]],

                        [token_to_id["statquest"],
                         token_to_id["is"],
                         token_to_id["what"],
                         token_to_id["<EOS>"],
                         token_to_id["awesome"]]])

labels = torch.tensor([[token_to_id["is"],
                        token_to_id["statquest"],
                        token_to_id["<EOS>"],
                        token_to_id["awesome"],
                        token_to_id["<EOS>"]],

                        [token_to_id["is"],
                        token_to_id["what"],
                        token_to_id["<EOS>"],
                        token_to_id["awesome"],
                        token_to_id["<EOS>"]]])

In [6]:
dataset = TensorDataset(inputs, labels)
dataloader = DataLoader(dataset)

In [7]:
def configure_optimizers(self):
    return Adam(self.parameters(), lr=0.1)
    
def training_step(self, batch, batch_idx):
    input_tokens, labels = batch
    output = self.forward(input_tokens[0])
    loss = self.loss(output, labels[0]) 
    return loss

In [8]:
model = DecoderOnlyTransformer(num_tokens=len(token_to_id), d_model=2, max_len=6)

In [ ]:
trainer = L.Trainer(max_epochs=30)
trainer.fit(model, train_dataloaders=dataloader)

In [10]:
model_input = torch.tensor([token_to_id["what"],
                            token_to_id["is"],
                            token_to_id["statquest"],
                            token_to_id["<EOS>"]])

input_length = model_input.size(dim=0)
predictions = model(model_input)
predicted_id = torch.tensor([torch.argmax(predictions[-1,:])])
print(predicted_id)
predicted_ids = predicted_id

max_length = 6
for i in range(input_length, max_length) :
    if (predicted_id == token_to_id["<EOS>"]):
        break
    
    model_input = torch.cat((model_input, predicted_id))

    predictions = model(model_input)
    predicted_id = torch.tensor([torch.argmax(predictions[-1,:])])
    predicted_ids = torch.cat((predicted_ids, predicted_id) )

print("Predicted Tokens:\n")
for id in predicted_ids:
    print("\t", id_to_token[id.item()])

tensor([0])
Predicted Tokens:

	 what
	 what
	 is
